# ARMA

We would like to apply ARMA model to predict our target variable. There is one problem: etr time-series is too short to conduct proper stationarity analysis. Based on wisdom of the experts and our knowledge about etr gained during EDA, we can assume that y variable is weakly stationary for every company. That's why we can apply ARMA model without d component! 

### Dependencies loading

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from pmdarima.arima import auto_arima
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import warnings

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 150)

# np.random.seed(1916) #uncomment if you want your code to be reproducible; for the purposes of our activity, let's add some randomness to the results

In [12]:
preprocessed_output_data_path = "../data/output"

df = pd.read_csv(f"{preprocessed_output_data_path}/train_fe.csv", index_col=0)
df_test = pd.read_csv(f"{preprocessed_output_data_path}/test_fe.csv", index_col=0)

### Dataset preparation

In [13]:
X = df[["Ticker", "etr"]]

In [14]:
Y = df_test[["Ticker", "etr"]]

### AutoArima model application

We will obtain best ARMA model in auto ARIMA procedure with HQIC information criterion. There is no need for CV here!

In [15]:
y_pred = list()
for j, i in enumerate(X.Ticker.unique()):
    stepwise_model = auto_arima(
        X[X.Ticker == i].etr.tolist(),
        start_p=1,
        start_q=1,
        max_p=3,
        max_q=3,
        m=1,
        start_P=0,
        seasonal=False,
        d=None,
        D=0,
        suppress_warnings=True,
        stationary=True,
        stepwise=True,
        information_criterion="hqic",
    )
    y_pred.append(stepwise_model.predict(n_periods=1)[0])
    if j % 50 == 0:
        print(j)

0
50
100
150
200
250
300
350


In [16]:
y_true = Y.etr.tolist()

In [17]:
rmse = np.sqrt(mean_squared_error(y_true, y_pred))

In [18]:
mae = mean_absolute_error(y_true, y_pred)

In [19]:
pd.DataFrame([rmse, mae]).T.rename(columns={0: "RMSE", 1: "MAE"})

,RMSE,MAE
0,0.115057,0.073336


### Saving final predictions

In [20]:
pd.DataFrame([rmse, mae]).T.rename(columns={0: "RMSE", 1: "MAE"}).to_csv(
    "../data/models_output/arma.csv"
)